# <center><font color='red'><u>Result Scraper F1<u></font></center>

<img src='./images.jpg' width=100%>

In [12]:
import pandas as pd 
import numpy as np 

import re
import requests
from bs4 import BeautifulSoup as bs

In [10]:
# 'https://www.formula1.com/en/results/2016/races' 

# creating year link

initial_links = []
for i in range(2015,2026):
    print(f'https://www.formula1.com/en/results/{i}/races')
    initial_links.append(f'https://www.formula1.com/en/results/{i}/races')

https://www.formula1.com/en/results/2015/races
https://www.formula1.com/en/results/2016/races
https://www.formula1.com/en/results/2017/races
https://www.formula1.com/en/results/2018/races
https://www.formula1.com/en/results/2019/races
https://www.formula1.com/en/results/2020/races
https://www.formula1.com/en/results/2021/races
https://www.formula1.com/en/results/2022/races
https://www.formula1.com/en/results/2023/races
https://www.formula1.com/en/results/2024/races
https://www.formula1.com/en/results/2025/races


In [15]:
# Create an empty DataFrame from the start
df = pd.DataFrame()

for initial_link in initial_links:
    # Step 1: Get the webpage of a race year
    r = requests.get(initial_link)
    if r.status_code != 200:
        print(f"Failed to fetch {initial_link}")
        continue

    webpage = bs(r.content, "html.parser")

    # Get the year (only 2015–2025)
    h1 = webpage.find("h1")
    if h1:
        match = re.search(r"\b(201[5-9]|202[0-5])\b", h1.get_text())
        year = match.group() if match else None
    else:
        year = None

    if not year:
        print(f"No valid year found in {initial_link}")
        continue

    # Get race countries and links together
    a_tags = webpage.find_all("a", class_="flex gap-px-10 items-center")
    countries = [tag.get_text(strip=True) for tag in a_tags]
    base_link = "https://www.formula1.com"
    race_links = [base_link + tag["href"] for tag in a_tags if tag.get("href")]

    # Get race dates
    dates = []
    for row in webpage.select("tr.Table-module_body-row__shKd-"):
        tds = row.find_all("td")
        try:
            date = tds[1].get_text(strip=True)
            dates.append(date)
        except IndexError:
            continue

    # Check consistency
    min_len = min(len(race_links), len(countries), len(dates))
    if min_len == 0:
        print(f"No races found for year {year}")
        continue

    # Step 2: Build DataFrames for each race and concatenate at once
    race_dfs = []  # Temporary list of DataFrames for this year
    for i in range(min_len):
        race_link = race_links[i]
        try:
            table = pd.read_html(race_link)[0]
        except ValueError:
            print(f"No table found at {race_link}")
            continue

        num_rows = len(table)
        table["Year"] = [year] * num_rows
        table["Country"] = [countries[i]] * num_rows
        table["Race Date"] = [dates[i]] * num_rows
        table["Race Link"] = [race_link] * num_rows

        race_dfs.append(table)

    # Concatenate this year's races and then to main df
    if race_dfs:
        df_year = pd.concat(race_dfs, ignore_index=True, sort=False)
        df = pd.concat([df, df_year], ignore_index=True, sort=False)

# Finished
print("Data collection complete!")
print(df.head())

Data collection complete!
  Pos.  No.               Driver               Team  Laps Time / Retired  \
0    1   44    Lewis HamiltonHAM           Mercedes  58.0    1:31:54.067   
1    2    6      Nico RosbergROS           Mercedes  58.0        +1.360s   
2    3    5  Sebastian VettelVET            Ferrari  58.0       +34.523s   
3    4   19      Felipe MassaMAS  Williams Mercedes  58.0       +38.196s   
4    5   12       Felipe NasrNAS     Sauber Ferrari  58.0       +95.149s   

   Pts.  Year                     Country Race Date  \
0  25.0  2015  Flag of AustraliaAustralia    15 Mar   
1  18.0  2015  Flag of AustraliaAustralia    15 Mar   
2  15.0  2015  Flag of AustraliaAustralia    15 Mar   
3  12.0  2015  Flag of AustraliaAustralia    15 Mar   
4  10.0  2015  Flag of AustraliaAustralia    15 Mar   

                                           Race Link  
0  https://www.formula1.com/en/results/2015/races...  
1  https://www.formula1.com/en/results/2015/races...  
2  https://www.formul

In [16]:
df.head(40)

,Pos.,No.,Driver,Team,Laps,Time / Retired,Pts.,Year,Country,Race Date,Race Link
0,1,44,Lewis HamiltonHAM,Mercedes,58.0,1:31:54.067,25.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
1,2,6,Nico RosbergROS,Mercedes,58.0,+1.360s,18.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
2,3,5,Sebastian VettelVET,Ferrari,58.0,+34.523s,15.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
3,4,19,Felipe MassaMAS,Williams Mercedes,58.0,+38.196s,12.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
4,5,12,Felipe NasrNAS,Sauber Ferrari,58.0,+95.149s,10.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
5,6,3,Daniel RicciardoRIC,Red Bull Racing Renault,57.0,+1 lap,8.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
6,7,27,Nico HulkenbergHUL,Force India Mercedes,57.0,+1 lap,6.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
7,8,9,Marcus EricssonERI,Sauber Ferrari,57.0,+1 lap,4.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
8,9,55,Carlos SainzSAI,STR Renault,57.0,+1 lap,2.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
9,10,11,Sergio PerezPER,Force India Mercedes,57.0,+1 lap,1.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...


In [19]:
df.tail(500)

,Pos.,No.,Driver,Team,Laps,Time / Retired,Pts.,Year,Country,Race Date,Race Link
4193,NC,31,Esteban OconOCO,Alpine Renault,0.0,DNF,0.0,2024,Flag of QatarQatar,01 Dec,https://www.formula1.com/en/results/2024/races...
4194,1,4,Lando NorrisNOR,McLaren Mercedes,58.0,1:26:33.291,25.0,2024,Flag of United Arab EmiratesAbu Dhabi,08 Dec,https://www.formula1.com/en/results/2024/races...
4195,2,55,Carlos SainzSAI,Ferrari,58.0,+5.832s,18.0,2024,Flag of United Arab EmiratesAbu Dhabi,08 Dec,https://www.formula1.com/en/results/2024/races...
4196,3,16,Charles LeclercLEC,Ferrari,58.0,+31.928s,15.0,2024,Flag of United Arab EmiratesAbu Dhabi,08 Dec,https://www.formula1.com/en/results/2024/races...
4197,4,44,Lewis HamiltonHAM,Mercedes,58.0,+36.483s,12.0,2024,Flag of United Arab EmiratesAbu Dhabi,08 Dec,https://www.formula1.com/en/results/2024/races...
...,...,...,...,...,...,...,...,...,...,...,...
4688,16,23,Alexander AlbonALB,Williams,58.0,+90.327s,0.0,2025,Flag of United Arab EmiratesAbu Dhabi,07 Dec,https://www.formula1.com/en/results/2025/races...
4689,17,6,Isack HadjarHAD,Racing Bulls,57.0,+1 lap,0.0,2025,Flag of United Arab EmiratesAbu Dhabi,07 Dec,https://www.formula1.com/en/results/2025/races...
4690,18,30,Liam LawsonLAW,Racing Bulls,57.0,+1 lap,0.0,2025,Flag of United Arab EmiratesAbu Dhabi,07 Dec,https://www.formula1.com/en/results/2025/races...
4691,19,10,Pierre GaslyGAS,Alpine,57.0,+1 lap,0.0,2025,Flag of United Arab EmiratesAbu Dhabi,07 Dec,https://www.formula1.com/en/results/2025/races...


In [20]:
df.to_csv('f1_race_results(2015_2025).csv',index=False)

In [21]:
df = pd.read_csv('f1_race_results.csv')
df.head()

,Pos.,No.,Driver,Team,Laps,Time / Retired,Pts.,Year,Country,Race Date,Race Link
0,1,44,Lewis HamiltonHAM,Mercedes,58.0,1:31:54.067,25.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
1,2,6,Nico RosbergROS,Mercedes,58.0,+1.360s,18.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
2,3,5,Sebastian VettelVET,Ferrari,58.0,+34.523s,15.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
3,4,19,Felipe MassaMAS,Williams Mercedes,58.0,+38.196s,12.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
4,5,12,Felipe NasrNAS,Sauber Ferrari,58.0,+95.149s,10.0,2015,Flag of AustraliaAustralia,15 Mar,https://www.formula1.com/en/results/2015/races...
